<a href="https://colab.research.google.com/github/dominikmeyer95/academic-output/blob/feature%2Fapplied_numerical_finance/applied_numerical_finance/estimating_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Numerical Finance

* Group: Dominik Meyer, Mikhail Borovkov, Brage Bakken, Emanuele Chiarini, Enrico Giannelli
* Assignment: 10 Finite difference approach for the Greeks

# Key Resources & Notes

* Explain theoretical dimension of estimation methods in 1.1
* Estimate each Greek with all methods available and show potential difference in 1.2/1.3
* ...

# Table of Contents

* 1 [Option Price Sensitivities](#chapter_1)
    * 1.1 [Estimation Methods](#section_1_1)
        * 1.1.1 [Closed Form Solutions](#subsection_1_1_1)
        * 1.1.2 [Naive Finite Difference Approximation](#subsection_1_1_2)
        * 1.1.3 [Monte Carlo Finite Difference Approximation](#subsection_1_1_3)
    * 1.2 [Greek Estimation](#section_1_2)
        * 1.2.1 [Delta: Option's Underlying Price Sensitivity](#subsection_1_2_1)
        * 1.2.2 [Vega: Option's Volatility Sensitivity](#subsection_1_2_2)
        * 1.2.3 [Theta: Option's Time Sensitivity](#subsection_1_2_3)
        * 1.2.4 [Rho: Option's Interest Rate Sensitivity](#subsection_1_2_4)
        * 1.2.5 [Gamma: Delta's Underlying Price Sensitivity](#subsection_1_2_5)

<a class="anchor" name="chapter_1"></a>
# 1 Option Price Sensitivities

In [1]:
# import required packages
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# define parameters
S = 100
K = 100
T = 1
r = 0.01
q=0
sigma = 0.30
n=1000
prices = np.arange(1, 200,1)
maturities = np.arange(0.01, 5, 0.01)
rates = np.arange(0.00, 1, 0.01)
volatilities = np.arange(0.00, 1, 0.01)
increments = [0.1, 0.5, 1, 1.5, 2.0, 2.5, 3.0]

In [3]:
def generate_path(S, K, T, r, q, sigma, direction='call', seed=False):
    """
    Function that generates the price of a european options.
    - S, K, T, r, q and sigma have the usual meaning
    - call should be True if that price is for a call option or False if it is 
    for a put option
    - seed determines whether we want to use a specific seed
    """
    # handle seed parameter
    if seed==True:
        np.random.seed(123)
    else:
        pass
    # calculate final stock price
    z = np.random.normal()
    ST = S*np.exp((r-q-(sigma**2)/2)*T + sigma*np.sqrt(T)*z)
    # calculate european call and put prices
    if direction=='call':
        c0 = np.exp(-r*T)*max(ST-K, 0)
        return c0
    elif direction=='put':
        p0 = np.exp(-r*T)*max(K-ST, 0)
        return p0
    else:
        pass

In [4]:
# build function for option pricing (merged from both beforehand)
def price_option(S, K, T, r, q, sigma, direction, type, method, seed=True, n=10000):
    
    # calculate option prices via black_scholes
    if method=='black_scholes':
        # calculate d1 and d2
        d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        # calculate european call and put prices
        if direction=='call' and type=='european':
            c = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
            return c
        elif direction=='put' and type=='european':
            p = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
            return p
        else:
            pass
    
    # calculate option prices via monte carlo simulation
    elif method=='monte_carlo':
        # handle seed parameter
        if seed==True:
            np.random.seed(123)
        else:
            pass
        # calculate european call and put prices
        if direction=='call' and type=='european':
            # calculate option prices
            prices = [generate_path(S, K, T, r, q, sigma, direction=direction, seed=False) for i in range(n)]
            # calculate average price
            c0 = np.mean(prices)
            return c0
        elif direction=='put' and type=='european':
            # calculate option prices
            prices = [generate_path(S, K, T, r, q, sigma, direction=direction, seed=False) for i in range(n)]
            # calculate average price
            p0 = np.mean(prices)
            return p0
        else:
            pass
    
    # break if/else
    else:
        pass

In [5]:
results = {}
results["bs_price"] = price_option(S=S, K=K, T=50, r=r, q=q, sigma=sigma, direction='call', type='european', method='black_scholes', seed=False)
results["mc_price"] = price_option(S=S, K=K, T=50, r=r, q=q, sigma=sigma, direction='call', type='european', method='monte_carlo', seed=False)
results

{'bs_price': 77.8419218528427, 'mc_price': 82.90468850698873}

In [6]:
# build function for greek estimation
def estimate_greeks(S, K, T, r, q, sigma, direction, option_type, pricing_method, greek, estimation_method, greek_seed=False, path_seed=False, n=1000, h=0.01):
    """
    Estimate option greeks using closed form or finite differences.
    """

    # define general pricing parameters
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    # estimate delta
    if greek == "delta":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call":
                # delta = norm.cdf(d1, 0, 1)
                delta = np.exp(-q*T)*norm.cdf(d1, 0, 1)
            elif direction == "put":
                # delta = -norm.cdf(-d1, 0, 1)
                delta = np.exp(-q*T)*(norm.cdf(d1, 0, 1)-1)
            else:
                pass
        # implement naive finite difference estimation
        elif estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S+h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S-h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                delta = (actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                delta = (forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                delta = (forward_increment - backward_increment)/(2*h)
            else:
                pass
        # implement monte carlo finite difference estimation
        elif estimation_method in ['backward_difference_mc', 'forward_difference_mc', 'central_difference_mc']:
            # define finite difference parameters
            if greek_seed == True:
                np.random.seed(123)
            else:
                pass
            paths = pd.DataFrame()
            paths['actual_price'] = [generate_path(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['forward_increment'] = [generate_path(S=S+h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['backward_increment'] = [generate_path(S=S-h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            # calculate finite differences
            if estimation_method == "backward_difference_mc":
                delta = (np.mean(paths['actual_price']) - np.mean(paths['backward_increment']))/h
            elif estimation_method == "forward_difference_mc":
                delta = (np.mean(paths['forward_increment']) - np.mean(paths['actual_price']))/h
            elif estimation_method == "central_difference_mc":
                delta = (np.mean(paths['forward_increment']) - np.mean(paths['backward_increment']))/(2*h)
            else:
                pass
        # break if/else
        else:
            pass
        return delta
    
    # estimate gamma
    elif greek == "gamma":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call" or direction == "put":
                # gamma = norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T))
                gamma = np.exp(-q*T)*norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T))
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S+h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            double_forward_increment = price_option(S=S+2*h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S-h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            double_backward_increment = price_option(S=S-2*h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                gamma = (actual_price - 2*backward_increment + double_backward_increment)/(h**2)
            elif estimation_method == "forward_difference":
                gamma = (double_forward_increment - 2*forward_increment + actual_price)/(h**2)
            elif estimation_method == "central_difference":
                gamma = (forward_increment - 2*actual_price + backward_increment)/(h**2)
            else:
                pass
        # implement monte carlo finite difference estimation
        elif estimation_method in ['backward_difference_mc', 'forward_difference_mc', 'central_difference_mc']:
            # define finite difference parameters
            if greek_seed == True:
                np.random.seed(123)
            else:
                pass
            paths = pd.DataFrame()
            paths['actual_price'] = [generate_path(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['forward_increment'] = [generate_path(S=S+h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['double_forward_increment'] = [generate_path(S=S+2*h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['backward_increment'] = [generate_path(S=S-h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['double_backward_increment'] = [generate_path(S=S-2*h, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            # calculate finite differences
            if estimation_method == "backward_difference_mc":
                gamma = (np.mean(paths['actual_price']) - 2*np.mean(paths['backward_increment']) + np.mean(paths['double_backward_increment']))/(h**2)
            elif estimation_method == "forward_difference_mc":
                gamma = (np.mean(paths['double_forward_increment']) - 2*np.mean(paths['forward_increment']) + np.mean(paths['actual_price']))/(h**2)
            elif estimation_method == "central_difference_mc":
                gamma = (np.mean(paths['forward_increment']) - 2*np.mean(paths['actual_price']) + np.mean(paths['backward_increment']))/(h**2)
            else:
                pass
        # break if/else
        else:
            pass
        return gamma
    
    # estimate vega
    elif greek == "vega":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call" or direction == "put":
                # vega = S*norm.pdf(d1, 0, 1)*np.sqrt(T)
                vega = S*np.exp(-q*T)*norm.pdf(d1, 0, 1)*np.sqrt(T)
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S, K=K, T=T, r=r, q=q, sigma=sigma+h, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S, K=K, T=T, r=r, q=q, sigma=sigma-h, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                vega = (actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                vega = (forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                vega = (forward_increment - backward_increment)/(2*h)
            else:
                pass
        # implement monte carlo finite difference estimation
        elif estimation_method in ['backward_difference_mc', 'forward_difference_mc', 'central_difference_mc']:
            # # define finite difference parameters
            if greek_seed == True:
                np.random.seed(123)
            else:
                pass
            paths = pd.DataFrame()
            paths['actual_price'] = [generate_path(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['forward_increment'] = [generate_path(S=S, K=K, T=T, r=r, q=q, sigma=sigma+h, direction=direction, seed=path_seed) for i in range(n)]
            paths['backward_increment'] = [generate_path(S=S, K=K, T=T, r=r, q=q, sigma=sigma-h, direction=direction, seed=path_seed) for i in range(n)]
            # calculate finite differences
            if estimation_method == "backward_difference_mc":
                vega = (np.mean(paths['actual_price']) - np.mean(paths['backward_increment']))/h
            elif estimation_method == "forward_difference_mc":
                vega = (np.mean(paths['forward_increment']) - np.mean(paths['actual_price']))/h
            elif estimation_method == "central_difference_mc":
                vega = (np.mean(paths['forward_increment']) - np.mean(paths['backward_increment']))/(2*h)
            else:
                pass
        # break if/else
        else:
            pass
        # times by 0.01 for sensitivity to 1% change in volatility
        return vega*0.01
    
    # estimate theta
    elif greek == "theta":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call":
                # theta = -(S*norm.pdf(d1, 0, 1)*sigma)/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
                theta = -(S*np.exp(-q*T)*norm.pdf(d1, 0, 1)*sigma)/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
            elif direction == "put":
                # theta = -(S*norm.pdf(d1, 0, 1)*sigma)/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
                theta = -(S*np.exp(-q*T)*norm.pdf(d1, 0, 1)*sigma)/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S, K=K, T=T+h, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S, K=K, T=T-h, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                theta = -(actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                theta = -(forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                theta = -(forward_increment - backward_increment)/(2*h)
            else:
                pass
        # implement monte carlo finite difference estimation
        elif estimation_method in ['backward_difference_mc', 'forward_difference_mc', 'central_difference_mc']:
            # # define finite difference parameters
            if greek_seed == True:
                np.random.seed(123)
            else:
                pass
            paths = pd.DataFrame()
            paths['actual_price'] = [generate_path(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['forward_increment'] = [generate_path(S=S, K=K, T=T+h, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['backward_increment'] = [generate_path(S=S, K=K, T=T-h, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            # calculate finite differences
            if estimation_method == "backward_difference_mc":
                theta = -(np.mean(paths['actual_price']) - np.mean(paths['backward_increment']))/h
            elif estimation_method == "forward_difference_mc":
                theta = -(np.mean(paths['forward_increment']) - np.mean(paths['actual_price']))/h
            elif estimation_method == "central_difference_mc":
                theta = -(np.mean(paths['forward_increment']) - np.mean(paths['backward_increment']))/(2*h)
            else:
                pass
        # break if/else
        else:
            pass
        # divide by 365 to get per day sensitivity
        return theta/365
    
    # estimate rho
    elif greek == "rho":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call":
                rho = K*T*np.exp(-r*T)*norm.cdf(d2, 0, 1)
            elif direction == "put":
                rho = -K*T*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S, K=K, T=T, r=r+h, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S, K=K, T=T, r=r-h, q=q, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                rho = (actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                rho = (forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                rho = (forward_increment - backward_increment)/(2*h)
            else:
                pass
        # implement monte carlo finite difference estimation
        elif estimation_method in ['backward_difference_mc', 'forward_difference_mc', 'central_difference_mc']:
            # # define finite difference parameters
            if greek_seed == True:
                np.random.seed(123)
            else:
                pass
            paths = pd.DataFrame()
            paths['actual_price'] = [generate_path(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['forward_increment'] = [generate_path(S=S, K=K, T=T, r=r+h, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            paths['backward_increment'] = [generate_path(S=S, K=K, T=T, r=r-h, q=q, sigma=sigma, direction=direction, seed=path_seed) for i in range(n)]
            # calculate finite differences
            if estimation_method == "backward_difference_mc":
                rho = (np.mean(paths['actual_price']) - np.mean(paths['backward_increment']))/h
            elif estimation_method == "forward_difference_mc":
                rho = (np.mean(paths['forward_increment']) - np.mean(paths['actual_price']))/h
            elif estimation_method == "central_difference_mc":
                rho = (np.mean(paths['forward_increment']) - np.mean(paths['backward_increment']))/(2*h)
            else:
                pass
        # break if/else
        else:
            pass
        # times by 0.01 for sensitivity to 1% change in interest rate
        return rho*0.01

In [7]:
# quick sanity check (1/3)
greeks={}
greeks["delta"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='closed_form', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["gamma"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='closed_form', greek_seed=False, path_seed=False, n=10000, h=0.5)
greeks["vega"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='closed_form', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["theta"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='closed_form', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["roh"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='closed_form', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks

{'delta': 0.5727317593030405,
 'gamma': 0.013076461848524422,
 'vega': 0.39229385545573264,
 'theta': -0.017351936757537507,
 'roh': 0.44904908466519977}

In [8]:
# quick sanity check (2/3)
greeks={}
greeks["delta"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='central_difference', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["gamma"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='central_difference', greek_seed=False, path_seed=False, n=10000, h=0.5)
greeks["vega"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='central_difference', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["theta"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='central_difference', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["roh"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='central_difference', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks

{'delta': 0.5727317557916223,
 'gamma': 0.013076273752631096,
 'vega': 0.39229201171001904,
 'theta': -0.017352143431319734,
 'roh': 0.449021459321461}

In [9]:
# quick sanity check (3/3)
greeks={}
greeks["delta"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["gamma"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=10000, h=0.5)
greeks["vega"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["theta"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks["roh"]=estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=10000, h=0.01)
greeks

{'delta': 13.973610771118938,
 'gamma': 0.9874325603377514,
 'vega': 0.46210733230813617,
 'theta': -0.02188605651840384,
 'roh': 0.42857214078497297}

<a class="anchor" name="section_1_1"></a>
## 1.1 Estimation Methods

<a class="anchor" name="subsection_1_1_1"></a>
### 1.1.1 Closed Form Solutions

* show analytical representation

<a class="anchor" name="subsection_1_1_2"></a>
### 1.1.2 Naive Finite Difference Approximation

* show numerical representation

<a class="anchor" name="subsection_1_1_3"></a>
### 1.1.3 Monte Carlo Finite Difference Approximation

<a class="anchor" name="section_1_2"></a>
## 1.2 Greek Estimation

<a class="anchor" name="subsection_1_2_1"></a>
### 1.2.1 Delta: Option's Underlying Price Sensitivity

In [10]:
# estimate greek
sim_results = pd.DataFrame()
sim_results['monte_carlo_fde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='forward_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['monte_carlo_cde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['closed_form'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='closed_form', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
final_results = pd.DataFrame()
final_results['closed_form'] = estimate_greeks(S=prices, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='closed_form', greek_seed=False, path_seed=False)
for h in increments:
    finite_difference = estimate_greeks(S=prices, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='central_difference', greek_seed=False, path_seed=False, h=h)
    estimation_error = final_results['closed_form'] - finite_difference
    final_results['error (h={})'.format(str(h))] = estimation_error
# create multiplot
fig = make_subplots(rows=3, cols=2, specs=[[{"secondary_y": True, "colspan": 2}, None],[{"secondary_y": False}, {"secondary_y": False}],[{"secondary_y": False}, {"secondary_y": False}]],
horizontal_spacing=0.10, vertical_spacing=0.15, subplot_titles=("naive cde estimate & errors over increments", "cde estimate over simulations", "last cde simulation estimate", "fde estimate over simulations", "last fde simulation estimate"))
fig.add_trace(go.Scatter(x=prices, y=final_results["closed_form"], mode='lines', line_color='springgreen', name="closed form", legendgroup='1'), row=1, col=1, secondary_y=False)
transp = 1.0
for h in increments:
    transp-=0.1
    fig.add_trace(go.Scatter(x=prices, y=final_results["error (h={})".format(str(h))], mode='lines', line_color='tomato', opacity=transp, name="error (h={})".format(str(h)), legendgroup='1'), row=1, col=1, secondary_y=True)
# create second subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_cde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='2'), row=2, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='2'), row=2, col=1)
# create third subplot
fig.add_trace(go.Bar(x=['monte_carlo_cde'], y=[sim_results['monte_carlo_cde'].iloc[-1]], name="monte carlo", legendgroup='2', marker_color='slateblue'), row=2, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='2', marker_color='yellow'), row=2, col=2)
# create fourth subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_fde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='3'), row=3, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='3'), row=3, col=1)
# create fifth subplot
fig.add_trace(go.Bar(x=['monte_carlo_fde'], y=[sim_results['monte_carlo_fde'].iloc[-1]], name="monte carlo", legendgroup='3', marker_color='slateblue'), row=3, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='3', marker_color='yellow'), row=3, col=2)
# format multiplot
fig.update_layout(template="plotly_dark", height=1200, title_x=0.5, showlegend=True, legend_tracegroupgap=280)
fig.update_xaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig['layout']['xaxis1']['title']='stock price'
fig['layout']['xaxis2']['title']='number of simulations'
fig['layout']['xaxis3']['title']='estimation method'
fig['layout']['xaxis4']['title']='number of simulations'
fig['layout']['xaxis5']['title']='estimation method'
# fig['layout']['xaxis4']['title']='Label x-axis 4'
fig.update_yaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig['layout']['yaxis1']['title']='delta level'
fig['layout']['yaxis3']['title']='delta level'
fig['layout']['yaxis4']['title']='delta level'
fig['layout']['yaxis5']['title']='delta level'
fig['layout']['yaxis6']['title']='delta level'
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys


<a class="anchor" name="subsection_1_2_2"></a>
### 1.2.2 Vega: Option's Volatility Sensitivity

In [11]:
# estimate greek
sim_results = pd.DataFrame()
sim_results['monte_carlo_fde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='forward_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['monte_carlo_cde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['closed_form'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='closed_form', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
final_results = pd.DataFrame()
final_results['closed_form'] = estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=volatilities, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='closed_form', greek_seed=False, path_seed=False)
for h in increments:
    finite_difference = estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=volatilities, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='central_difference', greek_seed=False, path_seed=False, h=h)
    estimation_error = final_results['closed_form'] - finite_difference
    final_results['error (h={})'.format(str(h))] = estimation_error

# create multiplot
fig = make_subplots(rows=3, cols=2, specs=[[{"secondary_y": True, "colspan": 2}, None],[{"secondary_y": False}, {"secondary_y": False}],[{"secondary_y": False}, {"secondary_y": False}]],
horizontal_spacing=0.10, vertical_spacing=0.15, subplot_titles=("naive cde estimate & errors over increments", "cde estimate over simulations", "last cde simulation estimate", "fde estimate over simulations", "last fde simulation estimate"))
fig.add_trace(go.Scatter(x=volatilities, y=final_results["closed_form"], mode='lines', line_color='springgreen', name="closed form", legendgroup='1'), row=1, col=1, secondary_y=False)
transp = 1.0
for h in increments:
    transp-=0.1
    fig.add_trace(go.Scatter(x=volatilities, y=final_results["error (h={})".format(str(h))], mode='lines', line_color='tomato', opacity=transp, name="error (h={})".format(str(h)), legendgroup='1'), row=1, col=1, secondary_y=True)
# create second subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_cde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='2'), row=2, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='2'), row=2, col=1)
# create third subplot
fig.add_trace(go.Bar(x=['monte_carlo_cde'], y=[sim_results['monte_carlo_cde'].iloc[-1]], name="monte carlo", legendgroup='2', marker_color='slateblue'), row=2, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='2', marker_color='yellow'), row=2, col=2)
# create fourth subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_fde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='3'), row=3, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='3'), row=3, col=1)
# create fifth subplot
fig.add_trace(go.Bar(x=['monte_carlo_fde'], y=[sim_results['monte_carlo_fde'].iloc[-1]], name="monte carlo", legendgroup='3', marker_color='slateblue'), row=3, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='3', marker_color='yellow'), row=3, col=2)
# format multiplot
fig.update_layout(template="plotly_dark", height=1200, title_x=0.5, showlegend=True, legend_tracegroupgap=280)
fig.update_xaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig['layout']['xaxis1']['title']='volatility'
fig['layout']['xaxis2']['title']='number of simulations'
fig['layout']['xaxis3']['title']='estimation method'
fig['layout']['xaxis4']['title']='number of simulations'
fig['layout']['xaxis5']['title']='estimation method'
# fig['layout']['xaxis4']['title']='Label x-axis 4'
fig.update_yaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig['layout']['yaxis1']['title']='vega level'
fig['layout']['yaxis3']['title']='vega level'
fig['layout']['yaxis4']['title']='vega level'
fig['layout']['yaxis5']['title']='vega level'
fig['layout']['yaxis6']['title']='vega level'
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in true_divide



<a class="anchor" name="subsection_1_2_3"></a>
### 1.2.3 Theta: Option's Time Sensitivity

In [12]:
# estimate greek
sim_results = pd.DataFrame()
sim_results['monte_carlo_fde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='forward_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['monte_carlo_cde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['closed_form'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='closed_form', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
final_results = pd.DataFrame()
final_results['closed_form'] = estimate_greeks(S=S, K=K, T=maturities, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='closed_form', greek_seed=False, path_seed=False)
for h in increments:
    finite_difference = estimate_greeks(S=S, K=K, T=maturities, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='central_difference', greek_seed=False, path_seed=False, h=h)
    estimation_error = final_results['closed_form'] - finite_difference
    final_results['error (h={})'.format(str(h))] = estimation_error

# create multiplot
fig = make_subplots(rows=3, cols=2, specs=[[{"secondary_y": True, "colspan": 2}, None],[{"secondary_y": False}, {"secondary_y": False}],[{"secondary_y": False}, {"secondary_y": False}]],
horizontal_spacing=0.10, vertical_spacing=0.15, subplot_titles=("naive cde estimate & errors over increments", "cde estimate over simulations", "last cde simulation estimate", "fde estimate over simulations", "last fde simulation estimate"))
fig.add_trace(go.Scatter(x=maturities, y=final_results["closed_form"], mode='lines', line_color='springgreen', name="closed form", legendgroup='1'), row=1, col=1, secondary_y=False)
transp = 1.0
for h in increments:
    transp-=0.1
    fig.add_trace(go.Scatter(x=maturities, y=final_results["error (h={})".format(str(h))], mode='lines', line_color='tomato', opacity=transp, name="error (h={})".format(str(h)), legendgroup='1'), row=1, col=1, secondary_y=True)
# create second subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_cde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='2'), row=2, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='2'), row=2, col=1)
# create third subplot
fig.add_trace(go.Bar(x=['monte_carlo_cde'], y=[sim_results['monte_carlo_cde'].iloc[-1]], name="monte carlo", legendgroup='2', marker_color='slateblue'), row=2, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='2', marker_color='yellow'), row=2, col=2)
# create fourth subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_fde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='3'), row=3, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='3'), row=3, col=1)
# create fifth subplot
fig.add_trace(go.Bar(x=['monte_carlo_fde'], y=[sim_results['monte_carlo_fde'].iloc[-1]], name="monte carlo", legendgroup='3', marker_color='slateblue'), row=3, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='3', marker_color='yellow'), row=3, col=2)
# format multiplot
fig.update_layout(template="plotly_dark", height=1200, title_x=0.5, showlegend=True, legend_tracegroupgap=280)
fig.update_xaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig['layout']['xaxis1']['title']='maturity'
fig['layout']['xaxis2']['title']='number of simulations'
fig['layout']['xaxis3']['title']='estimation method'
fig['layout']['xaxis4']['title']='number of simulations'
fig['layout']['xaxis5']['title']='estimation method'
# fig['layout']['xaxis4']['title']='Label x-axis 4'
fig.update_yaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig['layout']['yaxis1']['title']='theta level'
fig['layout']['yaxis3']['title']='theta level'
fig['layout']['yaxis4']['title']='theta level'
fig['layout']['yaxis5']['title']='theta level'
fig['layout']['yaxis6']['title']='theta level'
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in sqrt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in sqrt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in true_divide



<a class="anchor" name="subsection_1_2_4"></a>
### 1.2.4 Rho: Option's Interest Rate Sensitivity

In [13]:
# estimate greek
sim_results = pd.DataFrame()
sim_results['monte_carlo_fde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='forward_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['monte_carlo_cde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['closed_form'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='closed_form', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
final_results = pd.DataFrame()
final_results['closed_form'] = estimate_greeks(S=S, K=K, T=T, r=rates, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='closed_form', greek_seed=False, path_seed=False)
for h in increments:
    finite_difference = estimate_greeks(S=S, K=K, T=T, r=rates, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='central_difference', greek_seed=False, path_seed=False, h=h)
    estimation_error = final_results['closed_form'] - finite_difference
    final_results['error (h={})'.format(str(h))] = estimation_error

# create multiplot
fig = make_subplots(rows=3, cols=2, specs=[[{"secondary_y": True, "colspan": 2}, None],[{"secondary_y": False}, {"secondary_y": False}],[{"secondary_y": False}, {"secondary_y": False}]],
horizontal_spacing=0.10, vertical_spacing=0.15, subplot_titles=("naive cde estimate & errors over increments", "cde estimate over simulations", "last cde simulation estimate", "fde estimate over simulations", "last fde simulation estimate"))
fig.add_trace(go.Scatter(x=rates, y=final_results["closed_form"], mode='lines', line_color='springgreen', name="closed form", legendgroup='1'), row=1, col=1, secondary_y=False)
transp = 1.0
for h in increments:
    transp-=0.1
    fig.add_trace(go.Scatter(x=rates, y=final_results["error (h={})".format(str(h))], mode='lines', line_color='tomato', opacity=transp, name="error (h={})".format(str(h)), legendgroup='1'), row=1, col=1, secondary_y=True)
# create second subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_cde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='2'), row=2, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='2'), row=2, col=1)
# create third subplot
fig.add_trace(go.Bar(x=['monte_carlo_cde'], y=[sim_results['monte_carlo_cde'].iloc[-1]], name="monte carlo", legendgroup='2', marker_color='slateblue'), row=2, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='2', marker_color='yellow'), row=2, col=2)
# create fourth subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_fde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='3'), row=3, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='3'), row=3, col=1)
# create fifth subplot
fig.add_trace(go.Bar(x=['monte_carlo_fde'], y=[sim_results['monte_carlo_fde'].iloc[-1]], name="monte carlo", legendgroup='3', marker_color='slateblue'), row=3, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='3', marker_color='yellow'), row=3, col=2)
# format multiplot
fig.update_layout(template="plotly_dark", height=1200, title_x=0.5, showlegend=True, legend_tracegroupgap=280)
fig.update_xaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig['layout']['xaxis1']['title']='maturity'
fig['layout']['xaxis2']['title']='number of simulations'
fig['layout']['xaxis3']['title']='estimation method'
fig['layout']['xaxis4']['title']='number of simulations'
fig['layout']['xaxis5']['title']='estimation method'
# fig['layout']['xaxis4']['title']='Label x-axis 4'
fig.update_yaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig['layout']['yaxis1']['title']='rho level'
fig['layout']['yaxis3']['title']='rho level'
fig['layout']['yaxis4']['title']='rho level'
fig['layout']['yaxis5']['title']='rho level'
fig['layout']['yaxis6']['title']='rho level'
fig.show("")

<a class="anchor" name="subsection_1_2_5"></a>
### 1.2.5 Gamma: Delta's Underlying Price Sensitivity

In [14]:
# estimate greek
sim_results = pd.DataFrame()
sim_results['monte_carlo_fde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='forward_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['monte_carlo_cde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='central_difference_mc', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
sim_results['closed_form'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='closed_form', greek_seed=False, path_seed=False, n=i, h=0.01) for i in range(n)]
final_results = pd.DataFrame()
final_results['closed_form'] = estimate_greeks(S=prices, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='closed_form', greek_seed=False, path_seed=False)
for h in increments:
    finite_difference = estimate_greeks(S=prices, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='central_difference', greek_seed=False, path_seed=False, h=h)
    estimation_error = final_results['closed_form'] - finite_difference
    final_results['error (h={})'.format(str(h))] = estimation_error

# create multiplot
fig = make_subplots(rows=3, cols=2, specs=[[{"secondary_y": True, "colspan": 2}, None],[{"secondary_y": False}, {"secondary_y": False}],[{"secondary_y": False}, {"secondary_y": False}]],
horizontal_spacing=0.10, vertical_spacing=0.15, subplot_titles=("naive cde estimate & errors over increments", "cde estimate over simulations", "last cde simulation estimate", "fde estimate over simulations", "last fde simulation estimate"))
fig.add_trace(go.Scatter(x=prices, y=final_results["closed_form"], mode='lines', line_color='springgreen', name="closed form", legendgroup='1'), row=1, col=1, secondary_y=False)
transp = 1.0
for h in increments:
    transp-=0.1
    fig.add_trace(go.Scatter(x=prices, y=final_results["error (h={})".format(str(h))], mode='lines', line_color='tomato', opacity=transp, name="error (h={})".format(str(h)), legendgroup='1'), row=1, col=1, secondary_y=True)
# create second subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_cde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='2'), row=2, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='2'), row=2, col=1)
# create third subplot
fig.add_trace(go.Bar(x=['monte_carlo_cde'], y=[sim_results['monte_carlo_cde'].iloc[-1]], name="monte carlo", legendgroup='2', marker_color='slateblue'), row=2, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='2', marker_color='yellow'), row=2, col=2)
# create fourth subplot
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['monte_carlo_fde'], mode='lines', line_color='slateblue', name="monte carlo", legendgroup='3'), row=3, col=1)
fig.add_trace(go.Scatter(x=list(range(n)), y=sim_results['closed_form'], mode='lines', line_color='yellow', name="closed form", legendgroup='3'), row=3, col=1)
# create fifth subplot
fig.add_trace(go.Bar(x=['monte_carlo_fde'], y=[sim_results['monte_carlo_fde'].iloc[-1]], name="monte carlo", legendgroup='3', marker_color='slateblue'), row=3, col=2)
fig.add_trace(go.Bar(x=['closed_form'], y=[sim_results['closed_form'].iloc[-1]], name="closed form", legendgroup='3', marker_color='yellow'), row=3, col=2)
# format multiplot
fig.update_layout(template="plotly_dark", height=1200, title_x=0.5, showlegend=True, legend_tracegroupgap=280)
fig.update_xaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig['layout']['xaxis1']['title']='maturity'
fig['layout']['xaxis2']['title']='number of simulations'
fig['layout']['xaxis3']['title']='estimation method'
fig['layout']['xaxis4']['title']='number of simulations'
fig['layout']['xaxis5']['title']='estimation method'
# fig['layout']['xaxis4']['title']='Label x-axis 4'
fig.update_yaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig['layout']['yaxis1']['title']='gamma level'
fig['layout']['yaxis3']['title']='gamma level'
fig['layout']['yaxis4']['title']='gamma level'
fig['layout']['yaxis5']['title']='gamma level'
fig['layout']['yaxis6']['title']='gamma level'
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in log

